## Examples

The main purpose of this package is to add persistent data and program environment to AWS spot instances. AWS spot instances offer an 80% cost saving versus on-demand GPU instances. 

The examples here show how to setup and work with various types of server.

## Imports

In [2]:
from ipstartup import *
import aws
import server
import apps
from pdrive import Pdrive
import fabric.api as fab
from config import user, keyfile
fab.env.user = user
fab.output['everything'] = True

## Setup key and security group

It is assumed you already have an AWS account; config file; credentials file; and have requested a limit increase to access a GPU. This section just creates a key "key" and security group "simon".

In [3]:
# create a key
try:
    key = aws.ec2.create_key_pair(KeyName="key")
    with open(keyfile, "w") as f:
        f.write(key.key_material)
except Exception as e:
    log.warning(e)

[root:INFO]:starting (cellevents.py\30, time=19:21)
[root:WARNING]:An error occurred (InvalidKeyPair.Duplicate) when calling the CreateKeyPair operation: The keypair 'key' already exists. (<ipython-input-3-13fb431056c8>\7, time=19:21)


time: 1.8 s


In [4]:
# create a security group
try:
  sec = aws.ec2.create_security_group(GroupName="simon", 
                                Description="wordpress, jupyter, ssh")
  sec.authorize_ingress(
      IpPermissions=[dict(IpProtocol='tcp', FromPort=80, ToPort=80),
                     dict(IpProtocol='tcp', FromPort=443, ToPort=443),
                     dict(IpProtocol='tcp', FromPort=8888, ToPort=8888),
                     dict(IpProtocol='tcp', FromPort=22, ToPort=22)])
except Exception as e:
    log.warning(e)

[root:INFO]:starting (cellevents.py\30, time=19:21)
[root:WARNING]:An error occurred (InvalidGroup.Duplicate) when calling the CreateSecurityGroup operation: The security group 'simon' already exists for VPC 'vpc-b7e73cd3' (<ipython-input-4-2daf71c23845>\11, time=19:21)


time: 819 ms


## Create a new instance with a pdrive for programs and data

In [51]:
# create a server called "kate" with a free instance; and a volume called "fastai" mounted at /v1 to hold programs and data
server.create("kate", bootsize=None, itype="free", pdrive="fastai", pdrivesize=10)
apps.install_docker()
apps.set_docker_folder("/v1")

[root:INFO]:starting (cellevents.py\30, time=21:03)
[root:WARNING]:no snapshots found (pdrive.py\161, time=21:03)
[root:INFO]:waiting for instance running (server.py\64, time=21:03)
[root:INFO]:instance kate running at 34.248.174.161 (server.py\75, time=21:03)
[root:INFO]:waiting for ssh (server.py\138, time=21:03)
[root:INFO]:ssh connected (server.py\147, time=21:04)


[34.248.174.161] sudo: yum install docker -y
[34.248.174.161] out: Loaded plugins: priorities, update-motd, upgrade-helper
[34.248.174.161] out: Resolving Dependencies
[34.248.174.161] out: --> Running transaction check
[34.248.174.161] out: ---> Package docker.x86_64 0:1.12.6-1.17.amzn1 will be installed
[34.248.174.161] out: --> Processing Dependency: xfsprogs for package: docker-1.12.6-1.17.amzn1.x86_64
[34.248.174.161] out: --> Running transaction check
[34.248.174.161] out: ---> Package xfsprogs.x86_64 0:3.2.2-2.20.amzn1 will be installed
[34.248.174.161] out: --> Finished Dependency Resolution
[34.248.174.161] out: 
[34.248.174.161] out: Dependencies Resolved
[34.248.174.161] out: 
[34.248.174.161] out: ================================================================================
[34.248.174.161] out:  Package        Arch         Version                   Repository          Size
[34.248.174.161] out: ============================================================================

[root:INFO]:docker installed. if need to pull images then use ssh as this shows progress whereas fabric does not (apps.py\29, time=21:04)


AttributeError: 'Pdrive' object has no attribute 'formatdisk'

time: 1min 26s


In [61]:
pdrive.mount()

[root:INFO]:starting (cellevents.py\30, time=21:17)


[34.248.174.161] sudo: mkdir -p /v1
[34.248.174.161] sudo: mount /dev/xvdf /v1


[root:INFO]:volume mounted (pdrive.py\107, time=21:17)


[34.248.174.161] sudo: chown -R ec2-user:ec2-user /v1
time: 447 ms


In [62]:
# install some additional tools on /v1
fab.sudo("yum install -y -q git")
with fab.cd("/v1"):
  apps.install_github("simonm3", ["basics"])

[root:INFO]:starting (cellevents.py\30, time=21:18)


[34.248.174.161] sudo: yum install -y -q git
[34.248.174.161] out: Package git-2.7.4-1.47.amzn1.x86_64 already installed and latest version
[34.248.174.161] out: 

[34.248.174.161] sudo: if cd basics; then git pull; else git clone https://github.com/simonm3/basics.git basics; fi
[34.248.174.161] out: Already up-to-date.
[34.248.174.161] out: 

[34.248.174.161] put: C:\Users\s\Documents\py\apps\aws\..\basics\_creds.py -> /v1/basics/_creds.py
time: 2.29 s


In [ ]:
# manual steps as this easier to monitor progress
### ssh to server
### docker pull simonm3/fastai
### download data to /v1

In [67]:
# run fastai and add paths to additonal tools
apps.run_fastai()
with fab.quiet():
    fab.run("docker exec notebook python /host/basics/pathconfig.py")

[root:INFO]:starting (cellevents.py\30, time=21:22)


[34.248.174.161] sudo: docker run -v /v1:/host -w=/host/nbs -p 8888:8888 -d --restart=always --name fastai simonm3/fastai
[34.248.174.161] out: docker: Error response from daemon: Conflict. The name "/fastai" is already in use by container ea660804221f454e343d027ac09aea5810236aadab6da967da1fe58f729d0c42. You have to remove (or rename) that container to be able to reuse that name..
[34.248.174.161] out: See 'docker run --help'.
[34.248.174.161] out: 




Fatal error: sudo() received nonzero return code 125 while executing!

Requested: docker run -v /v1:/host -w=/host/nbs -p 8888:8888 -d --restart=always --name fastai simonm3/fastai
Executed: sudo -S -p 'sudo password:'  /bin/bash -l -c "docker run -v /v1:/host -w=/host/nbs -p 8888:8888 -d --restart=always --name fastai simonm3/fastai"

NoneType


Aborting.


SystemExit: 1

time: 829 ms


C:\Users\s\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


We can test the fastai notebook is running at the "kate" ip address port 8888. All of the setup time has used free instances and free storage. Next step is to terminate this instance. All data and programs will be preserved in a snapshot.

In [ ]:
server.terminate("kate")

## Working with an existing pdrive

In [ ]:
pdrive = Pdrive("cats")
pdrive.connect("sm1")

In [ ]:
pdrive.disconnect()

## Create more servers

In [ ]:
# create server with static ip address running wordpress
instance = server.create("sm1")
fab.env.host_string = aws.get_ips()[0]
aws.client.associate_address(InstanceId=instance.instance_id,
                             PublicIp=fab.env.host_string)
server.wait_ssh()
apps.install_docker()
apps.install_wordpress()

## Utilities

In [ ]:
# get a resource by name
aws.get("sm1")

In [ ]:
# get all resources (instances, volumes, snapshots)
aws.get(unique=False)

In [ ]:
# show instances used
aws.get_instances()

In [ ]:
# show python tasks running in containers
fab.env.host_string=aws.get("sm1").public_ip_address
server.get_tasks("python")

In [ ]:
apps.run_python("meetup")

In [ ]:
# show all tasks running in containers
server.get_tasks()

## Change docker location 

In [ ]:
# set to pdrive
apps.set_docker_folder("/v1")

In [ ]:
# set to boot drive
apps.set_docker_folder()